In [1]:
## import libaries
import pandas as pd
import numpy as np
import cv2
import os, sys
from tqdm import tqdm
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
def read_img(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    img = cv2.resize(img, (224,224))
    return img

In [3]:
train_consanant = pd.read_csv('Data/Consonant.csv')
train_vowel = pd.read_csv('Data/Vowel.csv')

In [4]:
TRAIN_PATH = 'Data/'

In [5]:
train_img = []
for img_path in tqdm(train_consanant['FileName'].values):
    train_img.append(read_img(TRAIN_PATH +"Consonant/"+ img_path))
for img_path in tqdm(train_vowel['FileName'].values):
    train_img.append(read_img(TRAIN_PATH +"Vowel/"+ img_path))    

100%|███████████████████████████████████████████████████████████████████████████████| 275/275 [00:02<00:00, 105.72it/s]


In [6]:
x_train = np.array(train_img, np.float32) / 255.

In [7]:
x_train.shape

(1430, 224, 224, 3)

In [8]:
y_train = [0]*1430

In [9]:
for x in range(1155,1430):
    y_train[x] = 1

In [10]:
y_train = np.array(y_train)
y_train = to_categorical(y_train)

In [11]:
from sklearn.cross_validation import train_test_split

In [12]:
X_train,X_test,y_train,y_test = train_test_split(x_train,y_train)

In [13]:
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D,Conv2D,BatchNormalization
from keras.layers import Activation, Dropout, Flatten, Dense

In [14]:
img_width, img_height = 224,224

input_shape = (img_width, img_height, 3)

model = Sequential()

model.add(Conv2D(96, kernel_size=(11, 11), strides=(4, 4), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(256, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(384, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(384, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(4096, activation='tanh'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='tanh'))
model.add(Dropout(0.5))

model.add(Dense(2, activation='softmax'))


In [15]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [16]:
nb_epoch = 10
nb_train_samples = 64

In [49]:
model.fit(X_train,y_train,batch_size=64,epochs=10,verbose=1,validation_data=(X_test,y_test))

Train on 1072 samples, validate on 358 samples
Epoch 1/10
1072/1072 [==============================] - 2s - loss: 0.0798 - acc: 0.9813 - val_loss: 0.6129 - val_acc: 0.9162
Epoch 2/10
1072/1072 [==============================] - 2s - loss: 0.0777 - acc: 0.9823 - val_loss: 0.3670 - val_acc: 0.9162
Epoch 3/10
1072/1072 [==============================] - 2s - loss: 0.0551 - acc: 0.9823 - val_loss: 0.5347 - val_acc: 0.9218
Epoch 4/10
1072/1072 [==============================] - 2s - loss: 0.0830 - acc: 0.9785 - val_loss: 0.6085 - val_acc: 0.9078
Epoch 5/10
1072/1072 [==============================] - 2s - loss: 0.0306 - acc: 0.9907 - val_loss: 0.3917 - val_acc: 0.9385
Epoch 6/10
1072/1072 [==============================] - 2s - loss: 0.0280 - acc: 0.9897 - val_loss: 0.3579 - val_acc: 0.9330
Epoch 7/10
1072/1072 [==============================] - 2s - loss: 0.0608 - acc: 0.9860 - val_loss: 0.6134 - val_acc: 0.8464
Epoch 8/10
1072/1072 [==============================] - 2s - loss: 0.0235 - ac

In [22]:
from sklearn.metrics import f1_score

In [50]:
y_pred = model.predict_classes(X_test)

352/358 [============================>.] - ETA: 0s

In [51]:
y_pred = to_categorical(y_pred)

In [52]:
print(f1_score(y_test,y_pred))

C:\Users\KAMALRAJ\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:756: DeprecationWarning: The default `weighted` averaging is deprecated, and from version 0.18, use of precision, recall or F-score with multiclass or multilabel data or pos_label=None will result in an exception. Please set an explicit value for `average`, one of (None, 'micro', 'macro', 'weighted', 'samples'). In cross validation use, for instance, scoring="f1_weighted" instead of scoring="f1".
  sample_weight=sample_weight)


0.955028308707


In [31]:
from sklearn.metrics import classification_report

In [53]:
print(classification_report(y_test,y_pred,target_names=['Consonant','Vowel']))

             precision    recall  f1-score   support

  Consonant       0.97      0.98      0.97       294
      Vowel       0.89      0.86      0.87        64

avg / total       0.95      0.96      0.96       358

